In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

# !git clone https://github.com/richardadiguna/dataset.git
# !git clone https://gitlab.com/yehezkielrchad/saved_model.git

# !cd saved_model/ && git status

''

In [104]:
# Upgrade the tensorflow library
!pip install --upgrade tensorflow

# Install matplotlib
!apt-get -qq install -y libfluidsynth1

# Install openCV
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

# Install Scikit-learn
!pip install -U scikit-learn

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages
Requirement already up-to-date: protobuf>=3.4.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: absl-py>=0.1.6 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: termcolor>=1.1.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: numpy>=1.13.3 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: six>=1.10.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: gast>=0.2.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: grpcio>=1.8.6 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: wheel>=0.26 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: astor>=0.6.0 in /usr/local/lib/pyth

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import os
import os.path
import random
from sklearn.model_selection import KFold, cross_val_score

In [0]:
def load_hand_dataset():
    listdir = os.listdir('/home/adiguna/Downloads/hand_dataset/training_dataset/training_data/cropt-images/')
    listdir.sort()
    counter = 1
    for filename in listdir:
        image_read = cv2.imread('/home/adiguna/Downloads/hand_dataset/training_dataset/training_data/cropt-images/' + filename)
        image_resize = cv2.resize(image_read, (72, 72))
        cv2.imwrite('/home/adiguna/Downloads/hand_dataset/training_dataset/training_data/test_resize_image_72x72/training_hand_image_' + str(counter) + '.jpg', image_resize)
        counter += 1


def load_train_data(volume):
    imagesPath = []
    labels = []
    for i in range(0, volume):
        imageFile = 'dataset/training-dataset-32x32-normal/training-data-' + str(i) + '.jpg'
        label = 1
        imagesPath.append(imageFile)
        labels.append(label)
    for i in range(0, volume):
        imageFile = 'dataset/training-dataset-zero-32x32-normal/training-data-zero-' + str(i) + '.jpg'
        label = 0
        imagesPath.append(imageFile)
        labels.append(label)
    return imagesPath, labels


def load_test_data(volume):
    imagesPath = []
    labels = []
    for i in range(0, volume):
        imageFile = 'dataset/test-dataset-32x32-normal/test-data-' + str(i) + '.jpg'
        label = 1
        imagesPath.append(imageFile)
        labels.append(label)
    return imagesPath, labels

In [0]:
def generate_random_data(images, labels, rand_index):
    image_array = []
    label_array = []
    for i in range(len(rand_index)):
        index = rand_index[i]
        image = images[index]
        label = labels[index]
        image_array.append(image)
        label_array.append(label)
    return image_array, label_array


def reformat(images, labels):
    image_array = []
    label_array = []
    for i in range(len(images)):
        image = cv2.imread(images[i])
        image_array.append(image)
        label = (np.arange(num_labels) == labels[i]).astype(np.float32).tolist()
#         label = labels[i]
        label_array.append(label)
    return image_array, label_array

In [0]:
def convolution2D(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def reluFunction(x):
    return tf.nn.relu(x)


def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


def local_response_normalization(conv, depth_radius, bias, alpha, beta, name):
    return tf.nn.lrn(conv, depth_radius, bias=bias, alpha=alpha / 9.0, beta=beta,
                     name=name)

In [0]:
# Input Variable
x_input = tf.placeholder(dtype=tf.float32, shape=None)
y_target = tf.placeholder(dtype=tf.float32, shape=None)
val_input = tf.placeholder(dtype=tf.float32, shape=None)
val_target = tf.placeholder(dtype=tf.float32, shape=None)
eval_input = tf.placeholder(dtype=tf.float32, shape=None)
eval_target = tf.placeholder(dtype=tf.float32, shape=None)
keep_prob = tf.placeholder(dtype=tf.float32, shape=None)

In [0]:
convolutional_1_output = 64
W_convolutional_1 = tf.Variable(tf.truncated_normal([5, 5, 3, convolutional_1_output], stddev=5e-2), name='W_convolutional_1')
b_convolutional_1 = tf.Variable(tf.zeros([convolutional_1_output]), name='b_convolutional_1')
convolutional_2_output = 64
W_convolutional_2 = tf.Variable(tf.truncated_normal([5, 5, convolutional_1_output, convolutional_2_output], stddev=5e-2), name='W_convolutional_2')
b_convolutional_2 = tf.Variable(tf.zeros([convolutional_2_output]), name='b_convolutional_2')
denseOutput1 = 384
denseInput1 = convolutional_2_output
W_dense_1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * denseInput1, denseOutput1], stddev=0.05), name='W_dense_1')
b_dense_1 = tf.Variable(tf.zeros([denseOutput1]), name='b_dense_1')
denseOutput2 = 192
denseInput2 = denseOutput1
W_dense_2 = tf.Variable(tf.truncated_normal(shape=[denseInput2, denseOutput2], stddev=0.05), name='W_dense_2')
b_dense_2 = tf.Variable(tf.zeros([denseOutput2]), name='b_dense_2')
denseOutput3 = 2
denseInput3 = denseOutput2
W_dense_3 = tf.Variable(tf.truncated_normal(shape=[denseInput3, denseOutput3], stddev=0.05), name='W_dense_3')
b_dense_3 = tf.Variable(tf.zeros([denseOutput3]), name='b_dense_3')

In [0]:
def fullyconnected_layer(convResult):
    finalConvShape = convResult.get_shape().as_list()
    finalShape = finalConvShape[1] * finalConvShape[2] * finalConvShape[3]
    flatConvResult = tf.reshape(convResult, shape=[-1, finalShape])
    dense_1 = tf.matmul(flatConvResult, W_dense_1) + b_dense_1
    relu_dense_1 = reluFunction(dense_1)
    dense_2 = tf.matmul(relu_dense_1, W_dense_2) + b_dense_2
    relu_dense_2 = reluFunction(dense_2)
    logits = tf.matmul(relu_dense_2, W_dense_3) + b_dense_3
    return logits

In [0]:
def convolution_network(left):
#   with tf.device('/gpu:0'):
  x = tf.reshape(left, shape=[-1, 32, 32, 3])
  convolution_1 = convolution2D(x, W_convolutional_1) + b_convolutional_1
  relu_convolution_1 = reluFunction(convolution_1)
  maxpool_convolution_1 = max_pool_3x3(relu_convolution_1)
  norm1 = local_response_normalization(maxpool_convolution_1, depth_radius=5, bias=1.0, alpha=0.001, beta=0.75, name='norm1')
  convolution_2 = convolution2D(norm1, W_convolutional_2) + b_convolutional_2
  relu_convolution_2 = reluFunction(convolution_2)
  norm2 = local_response_normalization(relu_convolution_2, depth_radius=5, bias=1.0, alpha=0.001, beta=0.75, name='norm2')
  maxpool_convolution_2 = max_pool_3x3(norm2)
  dense = fullyconnected_layer(maxpool_convolution_2)
  return dense

In [0]:
# Hyperparameter
learning_rate = 0.00001
batch_size = 32  # numbers of batch size must be declare during training process
steps = 125  # numbers of iteration must be declare during training process
epochs = 40  # numbers epochs must be declare during training process
num_labels = 2

In [0]:
global_step = tf.Variable(0, trainable=False, name='global_step', dtype= tf.int32)
learning_rate_decay_factor = 0.96 
num_epochs_per_decay = 12
num_examples_per_epochs = 4000
num_batches_per_epoch = int(num_examples_per_epochs / batch_size)
decay_steps = int(num_batches_per_epoch * num_epochs_per_decay)

In [0]:
model_learning_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps, learning_rate_decay_factor, staircase=True)

In [0]:
# L2 Normalization
beta = 0.04
regulizers_W_dense_1 = tf.nn.l2_loss(W_dense_1)
regulizers_W_dense_2 = tf.nn.l2_loss(W_dense_2)
regulizers = regulizers_W_dense_1 + regulizers_W_dense_2

In [0]:
# Define loss and optimizer
model_output = convolution_network(x_input)
validation_model_output = convolution_network(val_input)
test_model_output = convolution_network(eval_input)  # For evaluate the model

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model_output, labels=y_target))
regulizers = regulizers_W_dense_1 + regulizers_W_dense_2
cost = tf.reduce_mean(cost + beta * regulizers)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=model_learning_rate)
train_op = optimizer.minimize(loss=cost, global_step=global_step)

In [0]:
# Prediction using softmax
prediction = tf.nn.softmax(model_output)
validation_prediction = tf.nn.softmax(validation_model_output)
test_prediction = tf.nn.softmax(test_model_output)  # For evaluate the model

In [0]:
# Get Model Accuracy
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_target, 0))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
# Test accuracy model
def accuracy_of_batch(predictions, labels):
    targets = tf.squeeze(tf.cast(labels, tf.int32))
    targets_argmax = tf.cast(tf.argmax(targets, 1), tf.int32)
    bacth_predictions = tf.cast(tf.argmax(predictions, 1), tf.int32)
    predicted_correctly = tf.equal(bacth_predictions, targets_argmax)
    # print(sess.run(targets_argmax))
    # print(sess.run(bacth_predictions))
    # print(sess.run(predicted_correctly))
    accuracy = tf.reduce_mean(tf.cast(predicted_correctly, tf.float32)) * 100
    return accuracy
  
def accuracy_batch_sparse(predictions, labels):
  batch_prediction = tf.cast(tf.argmax(predictions, 1), tf.int32)
  predicted_correctly = tf.equal(batch_prediction, labels)
  accuracy = tf.reduce_mean(tf.cast(predicted_correctly, tf.float32))
  return accuracy

In [0]:
def k_fold(a, n_split):
    trainFolds = []
    validationFolds = []
    kf = KFold(n_splits=n_split)
    a = kf.split(a)
    for train_index, test_index in a:
        trainFolds.append(train_index)
        validationFolds.append(test_index)
    return trainFolds, validationFolds

def get_fold_data(data, index_array):
    result = []
    for index in index_array:
        result.append(data[index])
    return result

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
train_images, train_labels = load_train_data(2000)
train_images, train_labels = reformat(train_images, train_labels)

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.InteractiveSession()

In [0]:
# Validation accuracy
validation_accuracy = []
# Test Variable
test_pred = []
test_error = []
test_acc = []
# Train Variable
train_cost = []
train_cost_per_batch = []
train_acc = []
train_acc_per_batch = []
# Validation Variable
validation_pred = []
validation_error = []
validation_acc = []
validationTotalError = 0
# Validation per fold Variable
val_error_per_fold = []
val_pred_per_fold = []
val_total_error_per_fold = 0
# Batch Variable
numbers_of_cost_per_batch = 0
numbers_of_acc_per_batch = 0
sess.run(init)
index_distribution = len(train_images)

In [0]:
for i in range(0, epochs):
  train_cost_per_batch.clear()
  train_acc_per_batch.clear()
  for j in range(0, steps):
      rand_index = np.random.choice(index_distribution, size=batch_size)
      rand_train_images, rand_train_labels = generate_random_data(train_images, train_labels, rand_index)
      train = sess.run(train_op, feed_dict={x_input: rand_train_images, y_target: rand_train_labels})
      temp_train_pred = sess.run(prediction, feed_dict={x_input: rand_train_images})
      temp_train_cost = sess.run(cost, feed_dict={x_input: rand_train_images, y_target: rand_train_labels})
      temp_train_acc = sess.run(accuracy_of_batch(temp_train_pred, rand_train_labels))
      train_cost_per_batch.append(temp_train_cost)
      train_acc_per_batch.append(temp_train_acc)
      numbers_of_cost_per_batch += 1
      numbers_of_acc_per_batch += 1
      print("Training Validation hand image on epoch: " + str(i) + ", on step: " + str(j) + ', loss: ' + str(temp_train_cost) + ', acc: ' + str(temp_train_acc))
      if math.isnan(temp_train_cost):
          print('Loss is nan')
  accuracy_result_per_epochs = tf.reduce_mean(train_acc_per_batch)
  delta_cost_per_epochs = tf.reduce_mean(train_cost_per_batch)
  train_acc.append(accuracy_result_per_epochs)
  train_cost.append(delta_cost_per_epochs)
  print('delta_cost_per_epochs:' + str(sess.run(delta_cost_per_epochs)))
  print('accuracy_result_per_epochs: ' + str(sess.run(accuracy_result_per_epochs)))
  print('Learning rate: ', sess.run(model_learning_rate))
  if i % 5 == 0:
      print('Saving model......  ')
accuracyResult = (tf.reduce_mean(train_acc))
costResult = (tf.reduce_mean(train_cost))
print('total accuracy: ' + str(sess.run(accuracyResult)))
print('final cost: ' + str(sess.run(costResult)))
# save_path = saver.save(sess,'/saved_model/CUDA/fold_' + str(fold) + '/model_fold_' + str(fold) + '.ckpt')

In [0]:
test_images, test_labels = load_test_data(240)
test_images, test_labels = reformat(test_images, test_labels)
for i in range(0, len(test_images)):
  print('Test Model on test_image ' + str(i))
  test_image = test_images[i]
  test_label = [test_labels[i]]
  test_predict = sess.run(test_prediction, feed_dict={eval_input: test_image, keep_prob: 1.0})
  print(test_predict)
  error = sess.run(cost, feed_dict={x_input: test_images[i], y_target: test_label, keep_prob: 1.0})
  print(error)
  test_error.append(error)
  correct_pred_result = sess.run(correct_pred, feed_dict={prediction: test_predict, y_target: test_labels[i]})
  correct_pred_result = tf.cast(correct_pred_result, tf.float32)
  test_pred.append(correct_pred_result)
avg_error = tf.reduce_mean(test_error)
test_accuracy = tf.reduce_mean(tf.cast(test_pred, tf.float32))
print('Test Error Rate:', sess.run(avg_error))
print('Test Accuracy Result:', sess.run(test_accuracy))

In [0]:
sess.close()

In [0]:
!ls && cd saved_model/ && mkdir ZFNET && mkdir CUDA && cd ZFNET/ && mkdir fold_0